Hier werde ich viele verschiedene Skripte sammeln, die im Moment keinen Nutzen in einem von meinen anderen Notebooks haben. Evntl. kann ich die Sachen hier später noch gebrauchen.

In [ ]:
# Replace , with . in all files of a given paths

def get_abspath(basepath):
    """Get the files you need to import into your script with Path
    Returns a list of all filepaths (or folderpaths) of the files (or folders) in a repository.
    The repository is given this function with basepath.
    """
    df_list = []
    basepath = glob.glob(basepath)
    for entry in basepath:
        df_list.append(entry)
    return (df_list)

def replace_comma_with_dot_in_all_files(all_paths):
    """replace all "," with "." in all files of all_paths"""
    for path in all_paths:
        # Read in the file
        with open(path, 'r') as file :
            filedata = file.read()

        # Replace the target string
        filedata = filedata.replace(',', '.')

        # Write the file out again
        with open(path, 'w') as file:
            file.write(filedata)

import glob
all_paths=get_abspath(r".\path\to\the\files\with\kommas\*")
replace_comma_with_dot_in_all_files(all_paths)

In [ ]:
# Alte Version aus analyse_my_database_alsdorf.ipynb

# Check how similar the curves are at the watertank positions
# They should be very similar (equal in the range of error) at any other position too
####################I should implement calculation for this too!###############################
##################direkt funktion dafür erstellen#####################################

data=data_all_processed_shifted
diff_betweenChan_aftercorr_all_dates={}
# pairs: pair to compare the Temperature values
pairs=[["5","6"],["5","7"],["5","8"],["6","7"],["6","8"],["7","8"]]
for pair in pairs:
    f_ch=pair[0] #first channel
    s_ch=pair[1] #second_channel
    # for each channel the possible dates
    date_name_min_f_ch, date_iloc_min_f_ch = find_nearest_date(watertank_T_range_min,data[f_ch].index)
    date_name_max_f_ch, date_iloc_max_f_ch = find_nearest_date(watertank_T_range_max,data[f_ch].index)
    date_name_min_s_ch, date_iloc_min_s_ch = find_nearest_date(watertank_T_range_min,data[s_ch].index)
    date_name_max_s_ch, date_iloc_max_s_ch = find_nearest_date(watertank_T_range_max,data[s_ch].index)
    all_dates_in_range_channel_f_ch        = data[f_ch].index[date_iloc_min_f_ch:date_iloc_max_f_ch]
    all_dates_in_range_channel_s_ch        = data[s_ch].index[date_iloc_min_s_ch:date_iloc_max_s_ch]



##############################################################################
# date selection sit noch nicht optimal, da es sein kann, dass zwischendurch das Datum von einzelnen channels nicht
# im standart Muster ist, z.B. bei Measurement stops
# ich muss noch mehr checks einbauen die prüfen, dass bei Messungen wirklich nah zusammen sind
# mit den aktuellen Werten müsste es passen, von hand geprüft:
    #if f_ch == "7" and s_ch =="8":
    #    print(all_dates_in_range_channel_f_ch)
    #    print(all_dates_in_range_channel_s_ch)
    # second channel date is after first channel
    #if f_ch == "5" and s_ch =="6":
    #    all_dates_in_range_channel_s_ch=all_dates_in_range_channel_s_ch.drop(all_dates_in_range_channel_s_ch[0])
    
    # while all_dates_in_range_channel_f_ch[0] > all_dates_in_range_channel_s_ch[0]: 
    #     all_dates_in_range_channel_f_ch=all_dates_in_range_channel_f_ch.drop(all_dates_in_range_channel_f_ch[0])
    # #both have the same number of dates
    # while len(all_dates_in_range_channel_s_ch[0]) > len(all_dates_in_range_channel_f_ch[0]): 
    #     all_dates_in_range_channel_s_ch=all_dates_in_range_channel_s_ch.drop(all_dates_in_range_channel_s_ch[-1])
###########################################################################

    # Create dataframe for saving results
    diff_betweenChan_aftercorr = pd.DataFrame(columns=all_dates_in_range_channel_f_ch, index=watertank_len)
    diff_betweenChan_aftercorr.columns.names = ["Date f_ch"]
    diff_betweenChan_aftercorr.index.names   = ["Length [m]"]

    # go through all date pairs
    for date_name_f_ch, date_name_s_ch in zip(all_dates_in_range_channel_f_ch, all_dates_in_range_channel_s_ch):

        # check difference between the two dates
        allowed_difference = timedelta(minutes=13)
        time_diff = pd.to_datetime(date_name_f_ch) - date_name_s_ch
        if time_diff > timedelta(minutes=0):
            if time_diff > allowed_difference:
                print("Warning 1: large time difference")
                print(time_diff);print(f"{f_ch} {s_ch}")
        elif time_diff < timedelta(minutes=0):
            if time_diff * -1 >  allowed_difference:
                print("Warning 2: large time difference")
                print(time_diff * -1);print(f"{f_ch} {s_ch}")
        else: #val=0
            print("Something is wrong? No time diff between different channels!")

        # fill the dataframe for date pair
        for cable_length in diff_betweenChan_aftercorr.index[:2]: #excluding length which are only in chan 7 and 8
            # corrected Temp of a specific Date, Channel and watertank position
            val1 = data[f_ch][cable_length][date_name_f_ch]
            val2 = data[s_ch][cable_length][date_name_s_ch]
            diff_betweenChan_aftercorr[date_name_f_ch][cable_length] = val1 - val2

        if f_ch in ["7","8"] and s_ch in ["7","8"]: # calc diffs for length which are only in chan 7 and 8
            for cable_length in diff_betweenChan_aftercorr.index[2:]:
                # copied from above loop, just different cable_length
                val1 = data[f_ch][cable_length][date_name_f_ch]
                val2 = data[s_ch][cable_length][date_name_s_ch]
                diff_betweenChan_aftercorr[date_name_f_ch][cable_length] = val1 -val2

    # save dataframe for each date pair
    diff_betweenChan_aftercorr_all_dates[f"{f_ch} - {s_ch}"] = diff_betweenChan_aftercorr

# Calculate means for each pair over the date period
# Create Dataframe
diff_betweenChan_aftercorr_all_dates_means = pd.DataFrame(index=watertank_len,columns=diff_betweenChan_aftercorr_all_dates.keys())
diff_betweenChan_aftercorr_all_dates_means.index.names=["Length [m]"]
diff_betweenChan_aftercorr_all_dates_means.columns.names=["Channels"]
# Calculation
for pair in diff_betweenChan_aftercorr_all_dates.keys():
    for length in diff_betweenChan_aftercorr_all_dates_means.index:
        # mean value of all pair dates for length
        m_val = np.mean(abs(diff_betweenChan_aftercorr_all_dates[pair].loc[length]))
        diff_betweenChan_aftercorr_all_dates_means[pair][length] = m_val # save mean value

print("these are the means of the abs values of the differences between the shifted channel Temperature. Over the logging period of the T-Logger at the Watertank position")
diff_betweenChan_aftercorr_all_dates_means



